In [4]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

In [5]:
from rpy2.robjects.packages import importr

utils = importr("utils")
utils.chooseCRANmirror(ind=1)
#utils.install_packages('lme4')

%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [6]:
# Load data
conversations = pd.read_csv("results/reproduce_warlaumont/conversations.csv")

# convert True/False to 0/1:
conversations.replace({False: 0, True: 1}, inplace=True)

# normalize age
min_age, max_age = conversations["age"].min(), conversations["age"].max()
conversations["age"] = (conversations["age"] - min_age) / (max_age - min_age) * (1 - 0)

conversations.head()

,utterance_id,speaker_code,tokens,pos,age,corpus,transcript_file,child_name,speaker_code_next,start_time_next,...,response_speech_act,follow_up_transcript_raw,follow_up_start_time,follow_up_end_time,follow_up_is_speech_related,follow_up_is_intelligible,follow_up_speech_act,response_latency,response_latency_follow_up,has_response
0,300,CHI,"['sit', '.']",['v'],0.131579,Bernstein,/home/mitja/data/CHILDES/Bernstein/Children/Al...,Bernstein_Alice,MOT,29650.0,...,RT,all gone .,32175.0,39984.0,1,1,ST,0.0,2525.0,1
1,333,CHI,"['a', 'mommy', '.']","['det:art', 'n']",0.131579,Bernstein,/home/mitja/data/CHILDES/Bernstein/Children/Al...,Bernstein_Alice,MOT,138685.0,...,MK,eh &dɩdi .,140585.0,142428.0,1,0,YY,0.0,1900.0,1
2,336,CHI,['.'],['none'],0.131579,Bernstein,/home/mitja/data/CHILDES/Bernstein/Children/Al...,Bernstein_Alice,MOT,142428.0,...,ST,&sɩ &zi .,152302.0,153962.0,1,0,YY,0.0,9874.0,1
3,343,CHI,['.'],['none'],0.131579,Bernstein,/home/mitja/data/CHILDES/Bernstein/Children/Al...,Bernstein_Alice,MOT,153962.0,...,ST,&sɩ &zi .,173964.0,176230.0,1,0,YY,0.0,20002.0,1
4,350,CHI,['.'],['none'],0.131579,Bernstein,/home/mitja/data/CHILDES/Bernstein/Children/Al...,Bernstein_Alice,MOT,176230.0,...,YQ,&aɩa &la &næn &wæn .,200269.0,208049.0,1,0,YY,0.0,24039.0,1


## Quality of communicative feedback/ Caregiver contingency


### Timing:

In [11]:
%%R -i conversations
library(lme4)

m_caregiver_contingency<-glmer('has_response ~ utt_is_speech_related + age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: has_response ~ utt_is_speech_related + age + (1 | child_name)
   Data: conversations

      AIC       BIC    logLik  deviance  df.resid 
 346280.8  346324.5 -173136.4  346272.8    406206 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-20.9536   0.0793   0.4759   0.5534   1.6412 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 3.049    1.746   
Number of obs: 406210, groups:  child_name, 328

Fixed effects:
                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)           3.0507622  0.0001299   23487   <2e-16 ***
utt_is_speech_related 0.5580216  0.0001299    4296   <2e-16 ***
age                   1.1806747  0.0001299    9091   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) utt___
utt_s_spch_ 0.000      

## Effect of communicative feedback
### Positive Feedback: Timing

In [8]:
%%R -i conversations
library(lme4)

conversations_child_speech_related = subset(conversations, utt_is_speech_related==1)

m_child_contingency<-glmer('follow_up_is_speech_related ~ has_response + (1 | child_name)', data=conversations_child_speech_related, family=binomial)
print(summary(m_child_contingency))

R[write to console]: Loading required package: Matrix



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: follow_up_is_speech_related ~ has_response + (1 | child_name)
   Data: conversations_child_speech_related

     AIC      BIC   logLik deviance df.resid 
 83238.2  83270.8 -41616.1  83232.2   392526 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-49.583   0.060   0.153   0.165   1.202 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 3.696    1.923   
Number of obs: 392529, groups:  child_name, 322

Fixed effects:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   4.07875    0.16204   25.17  < 2e-16 ***
has_response  0.16043    0.02431    6.60 4.12e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
has_respons -0.134


In [10]:
%%R -i conversations
library(lme4)

conversations_child_speech_related = subset(conversations, utt_is_speech_related==1)

m_child_contingency<-glmer('follow_up_is_speech_related ~ has_response + age + (1 | child_name)', data=conversations_child_speech_related, family=binomial)
print(summary(m_child_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: follow_up_is_speech_related ~ has_response + age + (1 | child_name)
   Data: conversations_child_speech_related

     AIC      BIC   logLik deviance df.resid 
 83232.6  83276.1 -41612.3  83224.6   392525 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-47.885   0.060   0.156   0.166   1.183 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 3.537    1.881   
Number of obs: 392529, groups:  child_name, 322

Fixed effects:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   3.99347    0.15133  26.389  < 2e-16 ***
has_response  0.15700    0.02409   6.517 7.16e-11 ***
age           0.22066    0.07684   2.872  0.00408 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) hs_rsp
has_respons -0.148       
age         -0.164 -0.048
